In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score
from answer_judge.vqaEval import VQAEval

In [2]:
data_reader=ReadData('/root/autodl-tmp/hallscope/VQA/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,answers,question_id,most_likely,responses
0,[/root/autodl-fs/coco_images/train/COCO_train2...,[What kind of vehicles are pictured?\nAnswer t...,"[{'answer': ['car, truck', 'clash of titans', ...","[tensor(350077018), tensor(350077019), tensor(...",{'embedding': [[[ 0.01013184 0.00717163 -0.00...,None
1,[/root/autodl-fs/coco_images/train/COCO_train2...,[What does the man do standing against the wal...,"[{'answer': ['judge', '3', 'yes', '14a', 'post...","[tensor(264846000), tensor(264846001), tensor(...",{'embedding': [[[0.0022583 0.01220703 0.01611...,None
2,[/root/autodl-fs/coco_images/train/COCO_train2...,[What is the band logo name for this vehicle?\...,"[{'answer': ['derrick', 'derrick', 'derrick', ...","[tensor(96213008), tensor(96213009), tensor(96...",{'embedding': [[[-0.02246094 0.01843262 -0.00...,None
3,[/root/autodl-fs/coco_images/train/COCO_train2...,[What brand is the laptop?\nAnswer the questio...,"[{'answer': ['dell', 'signal', 'red', 'yes', '...","[tensor(262991002), tensor(394065000), tensor(...",{'embedding': [[[ 0.0100708 -0.00872803 -0.01...,None
4,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is this a paved road?\nAnswer the question us...,"[{'answer': ['no', 'road', 'yes', 'yes', 'yes'...","[tensor(219443002), tensor(219443003), tensor(...",{'embedding': [[[-0.00189972 0.01818848 -0.00...,None


In [5]:
df.shape

(2500, 6)

In [6]:
i=0

In [7]:
df.iloc[i]['img_path']

['/root/autodl-fs/coco_images/train/COCO_train2014_000000350077.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000350077.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000350077.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000350077.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000350077.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000350077.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000350077.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000350077.jpg']

In [8]:
df.iloc[i]['most_likely']['response']

['Truck',
 'Movie',
 'Theater',
 'No',
 'Coca cola',
 'Presidio',
 'Presidio',
 'Presidio']

In [9]:
df.iloc[i]['question']

['What kind of vehicles are pictured?\nAnswer the question using a single word or phrase.\n',
 'What event is being announced?\nAnswer the question using a single word or phrase.\n',
 'Where is this?\nAnswer the question using a single word or phrase.\n',
 'Is there a street light?\nAnswer the question using a single word or phrase.\n',
 'What is written on the truck?\nAnswer the question using a single word or phrase.\n',
 'What is the name of the establishment across the street?\nAnswer the question using a single word or phrase.\n',
 'What is the name of the theater?\nAnswer the question using a single word or phrase.\n',
 'What movie is playing at the theater?\nAnswer the question using a single word or phrase.\n']

In [10]:
df.iloc[i]['answers']

[{'answer': ['car, truck',
   'clash of titans',
   'outside',
   'no',
   'coca-cola',
   'presidio',
   'presidio',
   'clash of titans'],
  'answer_confidence': ['yes',
   'yes',
   'maybe',
   'yes',
   'yes',
   'yes',
   'yes',
   'yes'],
  'answer_id': tensor([1, 1, 1, 1, 1, 1, 1, 1])},
 {'answer': ['cars and trucks',
   'movie',
   'not sure',
   'no',
   'coca cola',
   'presidio',
   'presidio',
   'clash of titans'],
  'answer_confidence': ['yes',
   'yes',
   'no',
   'yes',
   'yes',
   'maybe',
   'yes',
   'maybe'],
  'answer_id': tensor([2, 2, 2, 2, 2, 2, 2, 2])},
 {'answer': ['cars and trucks',
   'clash of titans',
   'presidio',
   'yes',
   'coca cola',
   'presidio',
   'presidio',
   'clash of titans'],
  'answer_confidence': ['yes',
   'yes',
   'maybe',
   'yes',
   'maybe',
   'yes',
   'maybe',
   'yes'],
  'answer_id': tensor([3, 3, 3, 3, 3, 3, 3, 3])},
 {'answer': ['suv and coca cola truck',
   'movie',
   'city',
   'no',
   'coca-cola',
   'presidio',
   '

In [11]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [12]:
embeddings=np.concatenate(embeddings,axis=0)

In [13]:
embeddings.shape

(20000, 33, 4096)

In [14]:
def transpose_lines(x):
    f=pd.DataFrame(x)
    transposed=f['answer'].apply(pd.Series).T.values.tolist()
    return transposed

df['answers']=df['answers'].apply(transpose_lines)

In [15]:
gt_label=df['answers'].to_list()
gt_label=np.concatenate(gt_label,axis=0).tolist()

In [16]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
rsps=[]
for x in responses:
    rsps+=x

In [ ]:
rsps[0]

'Truck'

In [18]:
judge=VQAEval(gt_label,rsps)
judge.evaluate()
print(judge.evalQA)
print(judge.accuracy['overall'])

computing accuracy


  0%|          | 0/20000 [00:00<?, ?it/s]

100%|██████████| 20000/20000 [00:09<00:00, 2055.80it/s]

Done computing accuracy
{0: 0.0, 1: 100, 2: 0.0, 3: 100, 4: 100, 5: 100, 6: 100, 7: 0.0, 8: 0.0, 9: 100, 10: 100, 11: 0.0, 12: 0.0, 13: 100, 14: 100, 15: 100, 16: 100, 17: 0.0, 18: 100, 19: 100, 20: 100, 21: 100, 22: 100, 23: 100, 24: 100, 25: 100, 26: 100, 27: 100, 28: 33.33, 29: 100, 30: 100, 31: 33.33, 32: 100, 33: 0.0, 34: 100, 35: 100, 36: 100, 37: 100, 38: 100, 39: 100, 40: 33.33, 41: 100, 42: 100, 43: 100, 44: 33.33, 45: 100, 46: 100, 47: 100, 48: 100, 49: 100, 50: 100, 51: 100, 52: 66.67, 53: 100, 54: 100, 55: 100, 56: 0.0, 57: 33.33, 58: 100, 59: 100, 60: 100, 61: 100, 62: 100, 63: 100, 64: 100, 65: 100, 66: 100, 67: 100, 68: 33.33, 69: 100, 70: 100, 71: 100, 72: 66.67, 73: 100, 74: 100, 75: 0.0, 76: 100, 77: 0.0, 78: 100, 79: 100, 80: 100, 81: 100, 82: 100, 83: 100, 84: 0.0, 85: 0.0, 86: 0.0, 87: 100, 88: 100, 89: 100, 90: 100, 91: 100, 92: 100, 93: 33.33, 94: 100, 95: 100, 96: 100, 97: 100, 98: 66.67, 99: 100, 100: 33.33, 101: 100, 102: 100, 103: 100, 104: 33.33, 105: 33.33,

In [19]:
gt_scores=[judge.evalQA[i] for i in range(len(judge.evalQA))]

In [20]:
is_correct=[]
threshold=30
for i,x in enumerate(gt_scores):
    if x >=threshold:
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [21]:
is_correct.shape

(20000,)

In [21]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 20000 responses. 16833 answer correctly and 3167 answer wrongly


In [22]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 8000
# exclude validation samples.
train_idxs = train_val_idxs[:- validset_len]  # trainset index
val_idxs = train_val_idxs[- validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:8000
validset length:8000
testset length:4000
trainset postive: 6754 negative: 1246
valset postive: 6726 negative: 1274
testset postive: 3353 negative: 647


In [23]:
all_embeddings = embeddings
all_embeddings.shape

(20000, 33, 4096)

In [24]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [25]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [26]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(n_components,X[:,i_layer,:])
    scores=discriminator.get_score()
    return auroc(scores,y)
    #split=discriminator.get_best_split(y)
    #y_preds=(scores>split)
    #return accuracy_score(y,y_preds)

In [27]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,12),
    'layer':range(embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/11 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.5600151197513247
found best n_components:1,layer:3 Score: 0.634865824460574
found best n_components:1,layer:4 Score: 0.6350681835899117
found best n_components:1,layer:15 Score: 0.6394835570953833


  9%|▉         | 1/11 [00:13<02:12, 13.27s/it]

found best n_components:2,layer:4 Score: 0.6399913221309933
found best n_components:2,layer:15 Score: 0.6527674886601864


 27%|██▋       | 3/11 [00:40<01:48, 13.52s/it]

found best n_components:4,layer:6 Score: 0.6635362269521822


100%|██████████| 11/11 [02:27<00:00, 13.40s/it]


In [28]:
best_paras

{'n_components': 4, 'layer': 6, 'best_score': np.float64(0.6635362269521822)}

In [29]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [30]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(hall_label_val)

In [31]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_train[:,best_layer,:])
scores=discriminator.get_score()
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.65075 AUROC:0.6686438949916605


In [32]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.44325 AUROC:0.6673202295021967


Train Linear Prob to Get Better Results

In [33]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [34]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [35]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} f1:{f1_score(y_test,pred_test)}')

test set acc: 0.6605 f1:0.7707629979743417
